In [28]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import re

import seaborn as sns
from matplotlib             import pyplot as plt
from IPython.core.display   import HTML
from IPython.display        import Image

## Rent houses webscraping

In [1]:
get_houses_id('Berlin', get_pages_number('Berlin'))

In [56]:
# get all houses links from determinated city

# define a header
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# define City
cities = ['Berlin', 'Munich', 'Dusseldorf', 'Cologne', 'Dortmund', 'Frankfurt-am-Main', 'Nuremberg', 'Stuttgart', 'Hamburg', 'Hanover', 'Leipzig']

# list with dicts - houses infos 
houses_df_list = []

for city in cities:
    print(city)
    
    # define a url for each city
    url = f'https://housinganywhere.com/s/{city}--Germany/apartment-for-rent'    
    page = requests.get(url, headers=headers)

    # instance BS
    soup = BeautifulSoup(page.text, 'html.parser')

    # getting total pages number
    a = soup.find('div', class_ = 'MuiGrid-root jss208 jss207 MuiGrid-container MuiGrid-justify-xs-flex-end')
    try:
        try:
            b = a.findAll('a',class_ = 'MuiButtonBase-root MuiButton-root MuiButton-outlined jss209 MuiButton-outlinedSizeSmall MuiButton-sizeSmall')[3].span.text
            page_numbers = int(b)
        except:
            b = a.findAll('a',class_ = 'MuiButtonBase-root MuiButton-root MuiButton-outlined jss209 MuiButton-outlinedSizeSmall MuiButton-sizeSmall')[2].span.text
            page_numbers = int(b)
    except:
        pass

    # Crete a empty list for houses infos
    houses_links = []

    # going thru all pages from city
    for page in range(page_numbers):
        url = f'https://housinganywhere.com/s/{city}--Germany/apartment-for-rent?page={page}'
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        houses_list = soup.findAll('a', class_ = 'jss169 jss14')

        # going thru all houses from page
        for h in range(len(houses_list)):
            id = soup.findAll('a', class_ = 'jss169 jss14')[h].get('href').split('/')[2]
            comp = soup.findAll('a', class_ = 'jss169 jss14')[h].get('href').split('/')[-1]

            # get url from each house at houses list
            url = f'https://housinganywhere.com/room/{id}/de/{city}/{comp}'
            houses_links.append(url)

        houses_list.clear()

    # get houses infos link by link
    for house in houses_links:
        h = {}

        url = f'{str(house)}'

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)

        # Html 
        soup = BeautifulSoup(page.text, 'html.parser')

        # Hause ID
        try:
            h['house_id'] = url.split('/')[4]
        except:
            pass

        # City
        h['city'] = city

        # Location (coordinates? Street?)
        try:
            a = soup.findAll('h1', class_ = 'MuiTypography-root jss52 jss43 jss46 MuiTypography-h1 MuiTypography-displayBlock')
            h['street'] = a[0].text
        except:
            pass

        # Montly rent value
        try:
            h['montly_rent'] = soup.findAll('p', class_ = 'MuiTypography-root jss105 jss55 jss43 jss49 MuiTypography-h2 MuiTypography-displayBlock')[0].text
        except:
            pass

        # Deposit value
        try:
            if len(soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')) > 1:
                h['deposit_value'] = soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')[1].text
            else:
                h['deposit_value'] = soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-container MuiGrid-item MuiGrid-justify-xs-flex-end MuiGrid-grid-xs-6')[0].text
        except:
            pass

        # House rules - TRY TO FILTER "PET ALLOWED"
        try:
            h['house_rules'] = rules_list = []

            a =  soup.findAll('div', class_ = 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12')[1]

            b = a.findAll('div', class_ = 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-4')

            for iten in range(len(b)):
                i = a.findAll('div', class_ = 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-md-4')[iten].text
                rules_list.append(i)
        except:
            pass

        # Get apartment size
        try:
            h['size'] = soup.findAll('p', class_ = 'MuiTypography-root jss59 jss43 jss46 MuiTypography-body1 MuiTypography-displayInline')[1].text
        except:
            pass

        # Get lat/long
        try:
            a = soup.findAll('script', type = 'application/ld+json')[0].get_text
            h['lat_long'] = re.findall('(\d+\.\d+)', str(a))
        except:
            pass

        # add datetime
        h['scrapy_datetime'] = datetime.now().strftime('%Y/%m/%d - %H:%M:%S')

        # Append infos to houses list
        houses_df_list.append(h)

# Creating a DF
df_houses = pd.DataFrame(houses_df_list)

Berlin
Munich
Dusseldorf
Cologne
Dortmund
Frankfurt-am-Main
Nuremberg
Stuttgart
Hamburg
Hanover
Leipzig


In [57]:
df_houses.head()

,house_id,city,street,montly_rent,deposit_value,house_rules,size,lat_long,scrapy_datetime
0,1719697,Berlin,Nürnberger Straße,EUR 850,EUR 1700,"[Private living room, Private Toilet, Private ...",Property: 22 m2,"[52.50027, 13.33596]",2021/05/13 - 12:16:03
1,1716752,Berlin,Schnellerstraße,EUR 450,EUR 900,"[Shared toilet, Shared kitchen, Unisex bathroo...",Property: 90 m2,"[52.45495, 13.51636]",2021/05/13 - 12:16:05
2,1716755,Berlin,Schnellerstraße,EUR 475,EUR 950,"[Shared toilet, Shared kitchen, Unisex bathroo...",Property: 90 m2,"[52.45495, 13.51636]",2021/05/13 - 12:16:07
3,1724139,Berlin,Otto-Braun-Straße,EUR 760,EUR 500,"[Private facilities, Private Toilet, Private b...",Property: 25 m2,"[52.52449, 13.41863]",2021/05/13 - 12:16:09
4,1721869,Berlin,Otto-Braun-Straße,EUR 760,EUR 500,"[Private facilities, Private Toilet, Private b...",Property: 25 m2,"[52.52449, 13.41863]",2021/05/13 - 12:16:11


In [58]:
df_houses.isna().sum()

house_id            0
city                0
street              2
montly_rent        13
deposit_value       2
house_rules         0
size                2
lat_long            2
scrapy_datetime     0
dtype: int64

In [59]:
df_houses.shape

(1728, 9)

In [61]:
# Save to csv
# df_houses.to_csv('data/df_houses_full_raw.csv', index=False)

In [60]:
df_houses['city'].unique()

array(['Berlin', 'Munich', 'Dusseldorf', 'Cologne', 'Dortmund',
       'Frankfurt-am-Main', 'Nuremberg', 'Stuttgart', 'Hamburg',
       'Hanover', 'Leipzig'], dtype=object)

In [142]:
df_houses.head()

,Unnamed: 0,house_id,scrapy_datetime,city,street,size,pets,montly_rent,deposit_value,lat,long
0,0,1719697,2021/05/13 - 12:16:03,Berlin,Nürnberger Straße,22.0,Pets not allowed,850,1700,52.50027,13.33596
1,1,1716752,2021/05/13 - 12:16:05,Berlin,Schnellerstraße,90.0,Pets not allowed,450,900,52.45495,13.51636
2,2,1716755,2021/05/13 - 12:16:07,Berlin,Schnellerstraße,90.0,Pets not allowed,475,950,52.45495,13.51636
3,3,1724139,2021/05/13 - 12:16:09,Berlin,Otto-Braun-Straße,25.0,Pets not allowed,760,500,52.52449,13.41863
4,4,1721869,2021/05/13 - 12:16:11,Berlin,Otto-Braun-Straße,25.0,Pets not allowed,760,500,52.52449,13.41863


## Forquare API - nearby venues scraping.

In [76]:
# Set Forsquare API - client id and password 

CLIENT_ID = 'HERE GOES YOU FOURQUARE API ID' 
CLIENT_SECRET = 'HERE GOES YOU FOURQUARE API SECRET KEY' 
VERSION = '20210510' 
LIMIT = 100 


url = f'https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll=52.50027,13.33596&radius=300&limit={LIMIT}'

# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']

In [ ]:
venues_list=[]
for name, lat, long in zip(df_houses['house_id'], df_houses['lat'], df_houses['long']):
    #print(name)

    # create the API request URL
    url = f'https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius=500&limit={LIMIT}'

    # make the GET request
    try:
        results = requests.get(url).json()["response"]['groups'][0]['items']
    except:
        pass

    # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        long, 
        v['venue']['name'],
        v['venue']['location']['formattedAddress'][0],
        v['venue']['location']['formattedAddress'][1],
        v['venue']['categories'][0]['name'],
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['house_id', 
              'house_latitude', 
              'hoouse_longitude', 
              'venue',
              'venue_address',
              'city',
              'venue_category',
              'venue_latitude', 
              'venue_longitude']

In [ ]:
nearby_venues.isna().sum()

In [ ]:
nearby_venues

In [179]:
# save to a CSV file
nearby_venues.to_csv('data/nearby_venues_full_raw.csv', index=False)

In [185]:
results[0]['venue']['location']['formattedAddress'][1]

'50672 Köln'